![JohnSnowLabs](https://sparknlp.org/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp/blob/master/examples/python/transformers/openvino/HuggingFace_OpenVINO_Spark_NLP_MPNetForSequenceClassification.ipynb)

# Import OpenVINO MPNetForSequenceClassification models from HuggingFace 🤗 into Spark NLP 🚀

This notebook provides a detailed walkthrough on optimizing and exporting MPNetForSequenceClassification models from HuggingFace for use in Spark NLP, leveraging the various tools provided in the [Intel OpenVINO toolkit](https://www.intel.com/content/www/us/en/developer/tools/openvino-toolkit/overview.html) ecosystem.

Let's keep in mind a few things before we start 😊

- OpenVINO support was introduced in  `Spark NLP 5.4.0`, enabling high performance inference for models. Please make sure you have upgraded to the latest Spark NLP release.
- You can import models for MPNetForSequenceClassification from MPNetForSequenceClassification   and they have to be in `Text Classification` category.

## Export and Save HuggingFace model

- Let's install `transformers` package with the `openvino` extension and it's dependencies. You don't need `openvino` to be installed for Spark NLP, however, we need it to load and save models from HuggingFace.
- We lock `transformers` on version `4.34.1`. This doesn't mean it won't work with the future releases, but we wanted you to know which versions have been tested successfully.
- Additionally, we need to install `setfit` to load the model components.

In [1]:
!pip install -q --upgrade transformers[onnx]==4.35.1 optimum sentencepiece setfit
!pip install huggingface_hub==0.23.5
!pip install -q --upgrade openvino==2024.3
!pip install -q --upgrade optimum-intel==1.18.3
!pip install -q --upgrade onnx==1.12.0


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.1/123.1 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 3.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 2.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 44.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 453.7/453.7 kB 21.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 30.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.8/75.8 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 520.4/520.4 kB 24.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 77.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29

- We'll use [rodekruis/sml-ukr-message-classifier](https://huggingface.co/rodekruis/sml-ukr-message-classifier). As this is not a pure `transformers` model, we need to export the modules separately and combine them.

In [2]:
from setfit import SetFitModel
from transformers import AutoTokenizer

MODEL_NAME = "rodekruis/sml-ukr-message-classifier"
ONNX_MODEL = f"onnx_models/{MODEL_NAME}"

model = SetFitModel.from_pretrained(MODEL_NAME)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/655 [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/1.56k [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

model_head.pkl:   0%|          | 0.00/179k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:348: InconsistentVersionWarning: Trying to unpickle estimator LogisticRegression from version 1.2.2 when using version 1.3.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


## Exporting the Tokenizer

In [3]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, export=True)
tokenizer.save_pretrained(ONNX_MODEL)  # Save Tokenizer

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


('onnx_models/rodekruis/sml-ukr-message-classifier/tokenizer_config.json',
 'onnx_models/rodekruis/sml-ukr-message-classifier/special_tokens_map.json',
 'onnx_models/rodekruis/sml-ukr-message-classifier/vocab.txt',
 'onnx_models/rodekruis/sml-ukr-message-classifier/added_tokens.json',
 'onnx_models/rodekruis/sml-ukr-message-classifier/tokenizer.json')

Let's have a look inside these two directories and see what we are dealing with:

In [4]:
!ls -l {ONNX_MODEL}

total 932
-rw-r--r-- 1 root root    964 Sep 12 09:08 special_tokens_map.json
-rw-r--r-- 1 root root   1602 Sep 12 09:08 tokenizer_config.json
-rw-r--r-- 1 root root 710932 Sep 12 09:08 tokenizer.json
-rw-r--r-- 1 root root 231536 Sep 12 09:08 vocab.txt


- As you can see, we need to move `vocab.txt` to assets folder which Spark NLP will look for
- We also need `labels`. These are not contained in the model itself and we will have to fetch them manually. We will save this inside `labels.txt`

In [5]:
!mkdir -p {ONNX_MODEL}/assets
!mv {ONNX_MODEL}/vocab.txt {ONNX_MODEL}/assets/

In [6]:
!wget https://huggingface.co/{MODEL_NAME}/raw/main/label_dict.json

import json
# get label dictionary
with open("label_dict.json") as f:
    labels = json.load(f)

labels = [value for key, value in sorted(labels.items(), reverse=False, key=lambda x: int(x[0]))]

with open(ONNX_MODEL + "/assets/labels.txt", "w") as f:
    f.write("\n".join(labels))

--2024-09-12 09:08:12--  https://huggingface.co/rodekruis/sml-ukr-message-classifier/raw/main/label_dict.json
Resolving huggingface.co (huggingface.co)... 18.154.227.87, 18.154.227.69, 18.154.227.67, ...
Connecting to huggingface.co (huggingface.co)|18.154.227.87|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 589 [text/plain]
Saving to: ‘label_dict.json’

label_dict.json     100%[===================>]     589  --.-KB/s    in 0s      

2024-09-12 09:08:12 (344 MB/s) - ‘label_dict.json’ saved [589/589]



Voila! We have our `vocab.txt` and `labels.txt` inside assets directory

In [7]:
ls -l {ONNX_MODEL}/assets

total 232
-rw-r--r-- 1 root root    337 Sep 12 09:08 labels.txt
-rw-r--r-- 1 root root 231536 Sep 12 09:08 vocab.txt


## Combining and exporting the SetFit Modules

The `SetFitModel` is composed of these components, we need to export:

1. MPNet Embeddings Model
2. Pooling Module
3. Normalization Module
4. Prediction Module

We first create a custom torch module, to export it into a single ONNX graph.

In [8]:
import torch
from torch import nn

class SentencePredictor(nn.Module):
    def __init__(self, model):
        super().__init__()

        self.coeffs = torch.Tensor(model.model_head.coef_)
        self.intercept = torch.Tensor(model.model_head.intercept_)
        self.embeddings, self.pooling, self.normalize = model.model_body

    def predict(self, normed_embeddings):
        logits = normed_embeddings @ self.coeffs.T + self.intercept
        return logits

    def forward(self, input_ids, attention_mask):
        input = {"input_ids": input_ids, "attention_mask": attention_mask}
        embeddings_out = self.embeddings(input)
        pooling_out = self.pooling(embeddings_out)
        normalize_out = self.normalize(pooling_out)
        logits = self.predict(normalize_out["sentence_embedding"])
        return {"logits": logits}

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [9]:
sp = SentencePredictor(model)

In [10]:
input = model.model_body.tokenize(
    ["i loved the spiderman movie!", "pineapple on pizza is the worst 🤮"]
)

torch.onnx.export(
    sp,
    args=input,
    f=f"{ONNX_MODEL}/model.onnx",
    input_names=["input_ids", "attention_mask"],
    output_names=["logits"],
    dynamic_axes={
        "input_ids": {0: "batch_size", 1: "token_length"},
        "attention_mask": {0: "batch_size", 1: "token_length"},
        "logits": {0: "batch_size"},
    },
)

Now we have the model and all necessary files to import it into Spark NLP!

In [11]:
!ls -lR {ONNX_MODEL}

onnx_models/rodekruis/sml-ukr-message-classifier:
total 426464
drwxr-xr-x 2 root root      4096 Sep 12 09:08 assets
-rw-r--r-- 1 root root 435970222 Sep 12 09:08 model.onnx
-rw-r--r-- 1 root root       964 Sep 12 09:08 special_tokens_map.json
-rw-r--r-- 1 root root      1602 Sep 12 09:08 tokenizer_config.json
-rw-r--r-- 1 root root    710932 Sep 12 09:08 tokenizer.json

onnx_models/rodekruis/sml-ukr-message-classifier/assets:
total 232
-rw-r--r-- 1 root root    337 Sep 12 09:08 labels.txt
-rw-r--r-- 1 root root 231536 Sep 12 09:08 vocab.txt


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [12]:
import openvino as ov
model = ov.convert_model(f"{ONNX_MODEL}/model.onnx")

In [13]:
ov.save_model(model, 'openvino_model.xml')

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [14]:
!rm -rf {ONNX_MODEL}/model.onnx
!mv /content/openvino_model.bin {ONNX_MODEL}
!mv /content/openvino_model.xml {ONNX_MODEL}

## Import and Save MPNetForSequenceClassification in Spark NLP


- Let's install and setup Spark NLP in Google Colab
- This part is pretty easy via our simple script

In [ ]:
! wget http://setup.johnsnowlabs.com/colab.sh -O - | bash

--2024-01-10 17:00:06--  http://setup.johnsnowlabs.com/colab.sh
Resolving setup.johnsnowlabs.com (setup.johnsnowlabs.com)... 51.158.130.125
Connecting to setup.johnsnowlabs.com (setup.johnsnowlabs.com)|51.158.130.125|:80... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp/master/scripts/colab_setup.sh [following]
--2024-01-10 17:00:06--  https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp/master/scripts/colab_setup.sh
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1191 (1.2K) [text/plain]
Saving to: ‘STDOUT’

-                   100%[===================>]   1.16K  --.-KB/s    in 0s      

2024-01-10 17:00:06 (68.8 MB/s) - written to stdout [1191/1191]

Installing

Let's start Spark with Spark NLP included via our simple `start()` function

In [ ]:
import sparknlp

# let's start Spark with Spark NLP
spark = sparknlp.start()

print("Apache Spark version: {}".format(spark.version))

Apache Spark version: 3.2.3


- Let's use `loadSavedModel` functon in `MPNetForSequenceClassification` which allows us to load TensorFlow model in SavedModel format
- Most params can be set later when you are loading this model in `MPNetForSequenceClassification` in runtime like `setMaxSentenceLength`, so don't worry what you are setting them now
- `loadSavedModel` accepts two params, first is the path to the TF SavedModel. The second is the SparkSession that is `spark` variable we previously started via `sparknlp.start()`
- NOTE: `loadSavedModel` accepts local paths in addition to distributed file systems such as `HDFS`, `S3`, `DBFS`, etc. This feature was introduced in Spark NLP 4.2.2 release. Keep in mind the best and recommended way to move/share/reuse Spark NLP models is to use `write.save` so you can use `.load()` from any file systems natively.



In [ ]:
from sparknlp.annotator import *
from sparknlp.base import *

sequenceClassifier = (
    MPNetForSequenceClassification.loadSavedModel(ONNX_MODEL, spark)
    .setInputCols(["document", "token"])
    .setOutputCol("label")
)

- Let's save it on disk so it is easier to be moved around and also be used later via `.load` function

In [ ]:
sequenceClassifier.write().overwrite().save("./{}_spark_nlp_onnx".format(ONNX_MODEL))

Let's clean up stuff we don't need anymore

In [ ]:
!rm -rf {ONNX_MODEL}

Awesome 😎  !

This is your AlbertForSequenceClassification model from HuggingFace 🤗  loaded and saved by Spark NLP 🚀

In [ ]:
! ls -l {ONNX_MODEL}_spark_nlp_onnx

total 425832
drwxr-xr-x 4 root root      4096 Jan 10 17:13 fields
drwxr-xr-x 2 root root      4096 Jan 10 17:13 metadata
-rw-r--r-- 1 root root 436037492 Jan 10 17:14 MPNet_classification_onnx


Now let's see how we can use it on other machines, clusters, or any place you wish to use your new and shiny AlbertForSequenceClassification model 😊

In [ ]:
sequenceClassifier_loaded = (
    MPNetForSequenceClassification.load("./{}_spark_nlp_onnx".format(ONNX_MODEL))
    .setInputCols(["document", "token"])
    .setOutputCol("label")
)

You can see what labels were used to train this model via `getClasses` function:

In [ ]:
# .getClasses was introduced in spark-nlp==3.4.0
sequenceClassifier_loaded.getClasses()

['EDUCATION',
 'SHELTER',
 'PMER/NEWPROGRAMOPERTUNITIES',
 'TRANSPORT/CAR',
 'PAYMENTCVA',
 'PROGRAMINFO',
 'PSSRFL',
 'ARMY',
 'CHILDREN',
 'OTHERPROGRAMSOTHERNGOS',
 'CONNECTIVITY',
 'PROGRAMINFORMATION',
 'FOOD',
 'HEALTH',
 'TRANSLATION/LANGUAGE',
 'LEGAL',
 'PETS',
 'MONEY/BANKING',
 'SENTIMENT/FEEDBACK',
 'INCLUSIONCVA',
 'WORK/JOBS',
 'PARCEL',
 'TRANSPORT/MOVEMENT',
 'ANOMALY',
 'REGISTRATIONCVA',
 'WASH',
 'NFINONFOODITEMS',
 'GOODSSERVICES',
 'CONNECTWITHREDCROSS']

This is how you can use your loaded classifier model in Spark NLP 🚀 pipeline:

In [ ]:
from sparknlp.annotator import *
from sparknlp.base import *

document_assembler = DocumentAssembler().setInputCol("text").setOutputCol("document")

tokenizer = Tokenizer().setInputCols(["document"]).setOutputCol("token")

pipeline = Pipeline(stages=[document_assembler, tokenizer, sequenceClassifier_loaded])

# couple of simple examples
example = spark.createDataFrame([
    ["I love driving my car."],
    ["The next bus will arrive in 20 minutes."],
    ["pineapple on pizza is the worst 🤮"]
]).toDF("text")

result = pipeline.fit(example).transform(example)

# result is a DataFrame
result.select("text", "label.result").show()

+--------------------+--------------------+
|                text|              result|
+--------------------+--------------------+
|I love driving my...|     [TRANSPORT/CAR]|
|The next bus will...|[TRANSPORT/MOVEMENT]|
|pineapple on pizz...|              [FOOD]|
+--------------------+--------------------+



That's it! You can now go wild and use hundreds of `MPNetForSequenceClassification` models from HuggingFace 🤗 in Spark NLP 🚀
